In [8]:
from typing import TypedDict, Literal
import uuid

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, END
from langgraph.types import interrupt, Command

class State(TypedDict):
    llm_output: str
    decison : str

# Simulate an LLM output node
def llm_call_stimualtion(state: State) -> State:
    return {"llm_output": "This is generated output."}

# Human aporoval node

def human_approval_node(state: State) -> Command[Literal["approved_node", "rejected_node", "ended_node"]]:
    decision = interrupt(
        {
             "question": "Do you approve the following output?",
             "llm_output": state["llm_output"]
        }
    )
    if decision == 'approve':
        return Command(goto="approved_node", update={"decision": "approved"})
    elif decision == 'reject':
        return Command(goto="rejected_node", update={"decison": "rejected"})
    else :
        return Command(goto="ended_node", update={"decison": "ended"})

# Next steps after approval

def approved_node(state: State) -> State:
    print("Approved path taken.")
    return state

def rejected_node(state: State) -> State:
    print("Rejected path taken. ")
    return state

def ended_node(state: State):
    print ("This is end node")
    # return (Command(goto=END, update={"decision": ""}))
    return END

graph_builder = StateGraph(State)
graph_builder.add_node("llm_call_stimualtion", llm_call_stimualtion)
graph_builder.add_node("human_approval_node", human_approval_node)
graph_builder.add_node("approved_node", approved_node)
graph_builder.add_node("rejected_node", rejected_node)
graph_builder.add_node("ended_node", ended_node)

graph_builder.set_entry_point("llm_call_stimualtion")
graph_builder.add_edge("llm_call_stimualtion", "human_approval_node")
graph_builder.add_edge("approved_node", "human_approval_node" )
graph_builder.add_edge("rejected_node", "human_approval_node" )

memory = MemorySaver()
graph = graph_builder.compile(checkpointer = memory)

#pass a thread ID to the graph to run it.
config = {"configurable": {"thread_id":uuid.uuid4()}}
result = graph.invoke({},  config=config)
print(result["__interrupt__"])


final_result = graph.invoke(Command(resume="approve"), config=config)
# print(graph.invoke(Command(resume="Edited Text"), config=config))
print(final_result)



[Interrupt(value={'question': 'Do you approve the following output?', 'llm_output': 'This is generated output.'}, resumable=True, ns=['human_approval_node:863ed46a-1d24-1489-6015-bc8e69b33170'])]
Approved path taken.
{'llm_output': 'This is generated output.', '__interrupt__': [Interrupt(value={'question': 'Do you approve the following output?', 'llm_output': 'This is generated output.'}, resumable=True, ns=['human_approval_node:0e0b880a-ca7b-6158-f43f-64df0142ba45'])]}


In [9]:
final_result = graph.invoke(Command(resume="reject"), config=config)
print (final_result)


Rejected path taken. 
{'llm_output': 'This is generated output.', 'decison': 'rejected', '__interrupt__': [Interrupt(value={'question': 'Do you approve the following output?', 'llm_output': 'This is generated output.'}, resumable=True, ns=['human_approval_node:249f4ca3-3e46-f766-af5d-bc3dfb33b4ff'])]}


In [10]:
final_result = graph.invoke(Command(resume="approve"), config=config)
print (final_result)

Approved path taken.
{'llm_output': 'This is generated output.', 'decison': 'rejected', '__interrupt__': [Interrupt(value={'question': 'Do you approve the following output?', 'llm_output': 'This is generated output.'}, resumable=True, ns=['human_approval_node:dadafd95-f3f2-686d-2b00-0a0f200e9eb9'])]}


In [11]:
final_result = graph.invoke(Command(resume="ended"), config=config)
print (final_result)

This is end node


InvalidUpdateError: Expected dict, got __end__
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_GRAPH_NODE_RETURN_VALUE